## Homework FIve:  MyPython 1.2 -- Finishing the Function Language: Closures, Functions with Multiple Arguments, Lists, and (a little bit of ) Dynamic Error Checking

## General Homework Policies for CS 320

Please provide answers to these problems by filling in this notebook and submitting it via websubmit by <span style="color:red">Thursday 3/1 by midnight. You may submit it by Friday 3/2 midnight for a 10% penalty.</span> If the lab for this week involved a separate notebook, it is due at the same time. For any problems involving diagrams or drawings, you may try your hand at ASCII art or you may submit an electronic version of your drawings, either using drawing software or by scanning or photographing hand drawings. A message has been posted to Piazza showing you how to automatically include these in your notebook (a nice touch).  Make sure in the latter case that the result is legible; if we can not understand your drawing, it will receive no credit. 

There is a grace period, longer than 15 minutes but shorter than a couple of hours, after the deadlines, to account for temporarily problems with submission (dropped connections etc.). In cases where I have uploaded
a solution late on Wednesday night (probably close to the 15 minute mark!) no submissions will be accepted after the solution is uploaded. 

I will drop the lowest homework/lab combination at the end of the term.

For coding problems any <i> reasonable </i> solution, in which you understand the spirit of the exercise and try to learn this week's material through your efforts, will be graded based solely on the test cases. In general, shortcuts to avoid learning the material will be unreasonable. For example, you may not use any Python libraries or functions that make your task dramatically easier (e.g., you may use split(), replace(...), int(...), and float(...), as mentioned below, and any other normal functions, but may NOT use the Python library for regular expressions).  Good Python style, using list comprehensions, Numpy, creating helper functions in addition to what is required, etc., <i>when appropriate,</i> are quite reasonable! Check with me if you have questions about this. 
<p>
<span style="color:red"><b>Lab problems</b> may be worked on collaboratively at the lab and afterwards, with no need to indicate whom you collaborated with. (Of course, if you simply copy, then you will not learn what you need for homeworks and tests, and your grade will reflect your lack of effort.)</span> 

<span style="color:red"><b>Homework problems</b> must be solved individually, and will be subject to the College rules on plagiarism. <b>In particular, you are absolutely forbidden to copy code or answers from another student or from a source other than my slides and materials I provide to you.</b> We will run the code from notebooks through an electronic service that checks for plagiarism, and any violation will be dealt with harshly, up to and including an automatic drop of one letter grade at the end of the term (this policy will be familiar to those who took CS 111 at BU). Significant offences will be referred to the College Academic Misconduct Committee. These policies are necessary to provide a fair environment for you to do your best work with the expectation that the quality and quantity of your efforts will be rewarded appropriately. <b>Please take this as seriously as I do.</b></span> 

<b> Make sure to click Cell -> Run All before submitting notebooks!!</b>

### Lexical analyzer code. DO NOT MODIFY!

We have added a couple of new tokens for manipulating lists: nil, cons, first, and rest. 

In [1]:

# Token Categories -- slightly different than in HW01 and HW 02

ident = 'id'          # used in token as (indent, <string> )    
integer = 'int'       #  (integer, <value> )
floating = 'float'    #  (floating, <value> )  
trueTok = 'True'
falseTok = 'False'
noneTok = "None"
plus = '+'            #  (plus,)    rest as this one, no attribute necessary
minus = '-'          
mult = '*'
div = '/' 
mod = '%'
exp = '**'
lparen = '('
rparen = ')'
lbrace = '['
rbrace = ']'
equals = '=='
assign = '='
colon = ':'
lt = '<'
gt = '>'
ge = '>='
le = '<='
ne = '!='
andTok = 'and'
orTok = 'or'
notTok = 'not'
defTok = 'def'
semicolon = ';'
comma = ','
let = 'let'
inTok = 'in'
ifTok = 'if'
thenTok = 'then'
elseTok = 'else'
lambdaTok = 'lambda'
apply = 'apply'
returnTok = 'return'
nil = 'nil'
cons = 'cons'
first = 'first'
rest = 'rest'
error = 'error'       #  (error, <string> )     gives spelling of token that caused the lexical error

# Non-terminals are encoded as pairs using these codes:

S = 'S'             # (S,)  etc. 
E = 'E'
A = 'A'
L = 'L'
T = 'T'
F = 'F'
I = 'I'
C = 'C'
Top = 'top'
Bor = 'Bor'
Band = 'Band'
Bnot = 'Bnot'


# special token for end of input

end_of_input = '$'       # (end_of_input,) will be pretty-printed as ($,)


def tokenToString(t):
    if t == None:
        return str(t)
    elif t[0] in ['int','float','id']:
        return '(' + t[0] + ',' + str(t[1]) + ')'
    else:
        return '(' + t[0] + ',)'
        
def tokenListToString(lst):
    res = '[ '
    for t in lst[:-1]:
        res = res + tokenToString(t) + ', '
    res = res + tokenToString(lst[-1]) + ' ]'
    return res


# Code for lexer

# put white space between each separator or operator token and split into words

def splitTokens(s):
    for t in ['+','-','*','/','(',')','[',']',',','<', ':', '>', '=', '!','%']:
        s = s.replace(t,' ' + t + ' ')
    # now repair two-character tokens
    s = s.replace('<  =','<=')
    s = s.replace('>  =','>=')
    s = s.replace('=  =','==')
    s = s.replace('!  =','!=')
    s = s.replace('*  *','**')
    return s.split()

def isLetter(c):
    return 'a' <= c <= 'z' or 'A' <= c <= 'Z'

def isDigit(c):
    return '0' <= c <= '9' 

def isIdToken(s):
    state = 1
    for ch in s:
        if state == 1:
            if isLetter(ch) or ch == '_':
                state = 2
            else:
                return False
        elif state == 2:
            if isLetter(ch) or isDigit(ch) or ch == '_':
                state = 2
            else:
                return False
    return (state == 2)

                        
def isIntToken(s):
    if s == '0':
        return True
    state = 1
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            else:
                return False
    return (state == 2)
    
def isFloatToken(s):
    state = 1
    finalStates = [4,6]
    for ch in s:
        if state == 1:
            if isDigit(ch) and ch != '0':
                state = 2
            elif ch == '0':
                state = 3
            elif ch == '.':
                state = 5
            else:
                return False
        elif state == 2:
            if isDigit(ch):
                state = 2
            elif ch == '.':
                state = 4
            else:
                return False
        elif state == 3:
            if ch == '.':
                state = 4
            else:
                return False
        elif state == 4:
            if isDigit(ch):
                state = 4
            else:
                return False
        elif state == 5:
            if isDigit(ch):
                state = 6
            else:
                return False
        elif state == 6:
            if isDigit(ch):
                state = 6
            else:
                return False                
                
    return (state in finalStates)   
    
def isAssignToken(s):
    return s == '='

def isPlusToken(s):
    return s == '+'
    
def isMinusToken(s):
    return s == '-'

def isMultToken(s):
    return s == '*'
    
def isDivToken(s):
    return s == '/'
    
def isModToken(s):
    return s == '%'

def isExpToken(s):
    return s == '**'

def isEqualsToken(s):
    return s == '=='

def isNEToken(s):
    return s == '!='

def isLEToken(s):
    return s == '<='
    
def isGEToken(s):
    return s == '>='

def isLTToken(s):
    return s == '<'
    
def isGTToken(s):
    return s == '>'

def isLparenToken(s):
    return s == '('
    
def isRparenToken(s):
    return s == ')'

def isLbraceToken(s):
    return s == '['
    
def isRbraceToken(s):
    return s == ']'

def isColonToken(s):
    return s == ':'

def isSemicolonToken(s):
    return s == ';'

def isCommaToken(s):
    return s == ','

def isLetToken(s):
    return s == 'let'

def isLambdaToken(s):
    return s == 'lambda'

def isInToken(s):
    return s == 'in'

def isIfToken(s):
    return s == 'if'

def isThenToken(s):
    return s == 'then'

def isElseToken(s):
    return s == 'else'

def isDefToken(s):
    return s == 'def'

def isAndToken(s):
    return s == 'and'

def isOrToken(s):
    return s == 'or'

def isNotToken(s):
    return s == 'not'

def isReturnToken(s):
    return s == 'return'

def isTrueToken(s):
    return s == 'True'

def isFalseToken(s):
    return s == 'False'

def isNoneToken(s):
    return s == 'None'

def isNilToken(s):
    return s == 'nil'

def isConsToken(s):
    return s == 'cons'

def isFirstToken(s):
    return s == 'first'

def isRestToken(s):
    return s == 'rest'

def stringToToken(t):
    if isPlusToken(t):
        return (plus,) 
    elif isMinusToken(t):
        return (minus,) 
    elif isMultToken(t):
        return (mult,) 
    elif isDivToken(t):
        return (div,) 
    elif isExpToken(t):
        return (exp,) 
    elif isModToken(t):
        return (mod,) 
    elif isLTToken(t):
        return (lt,) 
    elif isGTToken(t):
        return (gt,) 
    elif isGEToken(t):
        return (ge,)    
    elif isLEToken(t):
        return (le,) 
    elif isEqualsToken(t):
        return (equals,)
    elif isNEToken(t):
        return (ne,)     
    elif isAndToken(t):
        return (andTok,) 
    elif isOrToken(t):
        return (orTok,)
    elif isNotToken(t):
        return (notTok,)    
    elif isLparenToken(t):
        return (lparen,) 
    elif isRparenToken(t):
        return (rparen,)
    elif isLbraceToken(t):
        return (lbrace,) 
    elif isRbraceToken(t):
        return (rbrace,)
    elif isDefToken(t):
        return (defTok,)    
    elif isAssignToken(t):
        return (assign,)
    elif isColonToken(t):
        return (colon,)
    elif isSemicolonToken(t):
        return (semicolon,)
    elif isCommaToken(t):
        return (comma,)
    elif isLetToken(t):
        return (let,)
    elif isLambdaToken(t):
        return (lambdaTok,)
    elif isIfToken(t):
        return (ifTok,)
    elif isThenToken(t):
        return (thenTok,)
    elif isElseToken(t):
        return (elseTok,)
    elif isInToken(t):
        return (inTok,t)
    elif isReturnToken(t):
        return (returnTok,t)
    elif isTrueToken(t):
        return (trueTok,)
    elif isFalseToken(t):
        return (falseTok,t)
    elif isNoneToken(t):
        return (noneTok,t)
    elif isNilToken(t):
        return (nil,)
    elif isConsToken(t):
        return (cons,)
    elif isFirstToken(t):
        return (first,)
    elif isRestToken(t):
        return (rest,)
    elif isIdToken(t):
        return (ident,t)
    elif isIntToken(t):
        return (integer,int(t)) 
    elif isFloatToken(t):
        return (floating,float(t)) 

    else:
        return (error,)


def lexer(s):
    return [stringToToken(t) for t in splitTokens(s)]


In [2]:
# Utilities for grammars

def rule_to_string(r):
    s = r[0][0] + ' := '
    for t in r[1]:
        s = s + t[0] + ' '
    return s
                          
def pprint_grammar(G): 
    for k in range(len(G)):
        print(str(k) + ": " + rule_to_string(G[k])) 
  
Grammar1_2 =    [(('S',),(('Top',),)),                  
         (('Top',),(('Assign',),)),                   
         (('Top',),(('Expr',),)),                       
         (('Top',),(('def',),('id',),('(',),(')',),(':',),('return',),('Expr',))),   
         (('Top',),(('def',),('id',),('(',),('id',),(')',),(':',),('return',),('Expr',))),   
         (('Top',),(('def',),('id',),('(',),('id',),(',',),('id',),(')',),(':',),('return',),('Expr',))),  
         (('Top',),(('def',),('id',),('(',),('id',),(',',),('id',),(',',),('id',),(')',),(':',),('return',),('Expr',))),      
         (('Assign',),(('id',),('=',),('Expr',))),         
         (('Expr',),(('if',),('Bor',),('then',),('Expr',),('else',),('Expr',))), 
         (('Expr',),(('lambda',),(':',),('Expr',))), 
         (('Expr',),(('lambda',),('id',),(':',),('Expr',))), 
         (('Expr',),(('lambda',),('id',),(',',),('id',),(':',),('Expr',))), 
         (('Expr',),(('lambda',),('id',),(',',),('id',),(',',),('id',),(':',),('Expr',))), 
         (('Expr',),(('let',),('Assign',),('in',),('Expr',))),       
         (('Expr',),(('Bor',),)), 
         (('Bor',),(('Bor',),('or',),('Band',))),   
         (('Bor',),(('Band',),)),                   
         (('Band',),(('Band',),('and',),('Bnot',))),
         (('Band',),(('Bnot',),)),                 
         (('Bnot',),(('not',),('Bnot',))),          
         (('Bnot',),(('C',),)),                     
         (('C',),(('E',),('==',),('E',))),          
         (('C',),(('E',),('!=',),('E',))),          
         (('C',),(('E',),('<',),('E',))),            
         (('C',),(('E',),('<=',),('E',))),          
         (('C',),(('E',),('>',),('E',))),            
         (('C',),(('E',),('>=',),('E',))),          
         (('C',),(('E',))),                         
         (('E',),(('E',),('+',),('T',))),           
         (('E',),(('E',),('-',),('T',))),           
         (('E',),(('T',),)),                       
         (('T',),(('T',),('*',),('F',))),          
         (('T',),(('T',),('/',),('F',))),          
         (('T',),(('T',),('%',),('F',))),          
         (('T',),(('F',),)),                       
         (('F',),(('-',),('F',))),                 
         (('F',),(('F1',),)),                       
         (('F1',),(('F2',), ('**',),('F1',))),     
         (('F1',),(('F2',),)),                                                                                   
         (('F2',),(('F2',), ('(',),(')',),) ),
         (('F2',),(('F2',), ('(',), ('Expr',), (')',),) ),
         (('F2',),(('F2',), ('(',), ('Expr',), (',',),('Expr',), (')',),) ),
         (('F2',),(('F2',), ('(',), ('Expr',), (',',),('Expr',), (',',),('Expr',), (')',),) ),
         (('F2',),(('id',),)),                      
         (('F2',),(('F3',),)), 
         (('F3',),(('int',),)),                    
         (('F3',),(('float',),)),                   
         (('F3',),(('True',),)),                   
         (('F3',),(('False',),)),  
         (('F3',),(('None',),)), 
         (('F3',),(('nil',),)), 
         (('F3',),(('cons',),)), 
         (('F3',),(('first',),)), 
         (('F3',),(('rest',),)), 
         (('F3',),(('(',),('Expr',),(')',))),
         (('F3',),(('[',),(']',))),
         (('F3',),(('[',),('EList',),(']',))),
         (('EList',),(('Expr',),(',',),('EList',))),      
         (('EList',),(('Expr',),)) ]        

pprint_grammar(Grammar1_2)

0: S := Top 
1: Top := Assign 
2: Top := Expr 
3: Top := def id ( ) : return Expr 
4: Top := def id ( id ) : return Expr 
5: Top := def id ( id , id ) : return Expr 
6: Top := def id ( id , id , id ) : return Expr 
7: Assign := id = Expr 
8: Expr := if Bor then Expr else Expr 
9: Expr := lambda : Expr 
10: Expr := lambda id : Expr 
11: Expr := lambda id , id : Expr 
12: Expr := lambda id , id , id : Expr 
13: Expr := let Assign in Expr 
14: Expr := Bor 
15: Bor := Bor or Band 
16: Bor := Band 
17: Band := Band and Bnot 
18: Band := Bnot 
19: Bnot := not Bnot 
20: Bnot := C 
21: C := E == E 
22: C := E != E 
23: C := E < E 
24: C := E <= E 
25: C := E > E 
26: C := E >= E 
27: C := E 
28: E := E + T 
29: E := E - T 
30: E := T 
31: T := T * F 
32: T := T / F 
33: T := T % F 
34: T := F 
35: F := - F 
36: F := F1 
37: F1 := F2 ** F1 
38: F1 := F2 
39: F2 := F2 ( ) 
40: F2 := F2 ( Expr ) 
41: F2 := F2 ( Expr , Expr ) 
42: F2 := F2 ( Expr , Expr , Expr ) 
43: F2 := id 
44: F2 := F3 
45: F3

#   Parser 

### The next two cells contain provided code for the homework; DO NOT MODIFY!

Changes from the last time are the table for the new grammar, in which the columns are indexed by strings and now err = None and Accept = 0. You should
not have to know this, but just in case....


In [3]:
# DFA to drive the shift-reduce parser

# The parser DFA will tell you what to do when the stack is
# in a given configuration.  A positive number gives the
# state transition, a non-positive number k is a reduction by the
# rule -k, i.e., -4 means a reduction by rule 4 with no
# advance in the input string.  None is an error and
# 0 is accept. 

err = None
accept = 0

table = {
  0:{'<=': None, 'F2': 1, '>': None, 'E': 2, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 12, 'and': None, '**': None, 'lambda': 13, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': 16, 'id': 17, 'def': 18, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 23, 'Band': 24, 'int': 26, '==': None, 'first': 25, 'Assign': 27, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  1:{'<=': -38, 'F2': None, '>': -38, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': 32, '=': None, '$': -38, ')': -38, ',': -38, 'S': None, '/': -38, '+': -38, ']': -38, 'then': -38, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -38, '<': -38, 'F': None, 'if': None, 'T': None, 'and': -38, '**': 33, 'lambda': None, '-': -38, '>=': -38, 'in': -38, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -38, '!=': -38, '*': -38, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -38, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -38, 'not': None},
  2:{'<=': 34, 'F2': None, '>': 36, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -27, ')': -27, ',': -27, 'S': None, '/': None, '+': 35, ']': -27, 'then': -27, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -27, '<': 41, 'F': None, 'if': None, 'T': None, 'and': -27, '**': None, 'lambda': None, '-': 38, '>=': 39, 'in': -27, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -27, '!=': 37, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': 40, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  3:{'<=': -53, 'F2': None, '>': -53, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -53, '=': None, '$': -53, ')': -53, ',': -53, 'S': None, '/': -53, '+': -53, ']': -53, 'then': -53, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -53, '<': -53, 'F': None, 'if': None, 'T': None, 'and': -53, '**': -53, 'lambda': None, '-': -53, '>=': -53, 'in': -53, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -53, '!=': -53, '*': -53, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -53, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -53, 'not': None},
  4:{'<=': -46, 'F2': None, '>': -46, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -46, '=': None, '$': -46, ')': -46, ',': -46, 'S': None, '/': -46, '+': -46, ']': -46, 'then': -46, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -46, '<': -46, 'F': None, 'if': None, 'T': None, 'and': -46, '**': -46, 'lambda': None, '-': -46, '>=': -46, 'in': -46, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -46, '!=': -46, '*': -46, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -46, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -46, 'not': None},
  5:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 47, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  6:{'<=': -36, 'F2': None, '>': -36, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -36, ')': -36, ',': -36, 'S': None, '/': -36, '+': -36, ']': -36, 'then': -36, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -36, '<': -36, 'F': None, 'if': None, 'T': None, 'and': -36, '**': None, 'lambda': None, '-': -36, '>=': -36, 'in': -36, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -36, '!=': -36, '*': -36, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -36, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -36, 'not': None},
  7:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -20, ')': -20, ',': -20, 'S': None, '/': None, '+': None, ']': -20, 'then': -20, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -20, '<': None, 'F': None, 'if': None, 'T': None, 'and': -20, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -20, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -20, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  8:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -18, ')': -18, ',': -18, 'S': None, '/': None, '+': None, ']': -18, 'then': -18, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -18, '<': None, 'F': None, 'if': None, 'T': None, 'and': -18, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -18, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -18, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  9:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 50, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': 49, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  10:{'<=': -34, 'F2': None, '>': -34, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -34, ')': -34, ',': -34, 'S': None, '/': -34, '+': -34, ']': -34, 'then': -34, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -34, '<': -34, 'F': None, 'if': None, 'T': None, 'and': -34, '**': None, 'lambda': None, '-': -34, '>=': -34, 'in': -34, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -34, '!=': -34, '*': -34, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -34, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -34, 'not': None},
  11:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 51, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  12:{'<=': -30, 'F2': None, '>': -30, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -30, ')': -30, ',': -30, 'S': None, '/': 52, '+': -30, ']': -30, 'then': -30, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -30, '<': -30, 'F': None, 'if': None, 'T': None, 'and': -30, '**': None, 'lambda': None, '-': -30, '>=': -30, 'in': -30, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -30, '!=': -30, '*': 53, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -30, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': 54, 'not': None},
  13:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 55, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 56, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  14:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 57, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  15:{'<=': -44, 'F2': None, '>': -44, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -44, '=': None, '$': -44, ')': -44, ',': -44, 'S': None, '/': -44, '+': -44, ']': -44, 'then': -44, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -44, '<': -44, 'F': None, 'if': None, 'T': None, 'and': -44, '**': -44, 'lambda': None, '-': -44, '>=': -44, 'in': -44, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -44, '!=': -44, '*': -44, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -44, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -44, 'not': None},
  16:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': 0, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  17:{'<=': -43, 'F2': None, '>': -43, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -43, '=': 58, '$': -43, ')': -43, ',': -43, 'S': None, '/': -43, '+': -43, ']': -43, 'then': -43, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -43, '<': -43, 'F': None, 'if': None, 'T': None, 'and': -43, '**': -43, 'lambda': None, '-': -43, '>=': -43, 'in': -43, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -43, '!=': -43, '*': -43, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -43, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -43, 'not': None},
  18:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 59, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  19:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': 60, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': 61, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 62, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  20:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -14, ')': -14, ',': -14, 'S': None, '/': None, '+': None, ']': -14, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': 63, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -14, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -14, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  21:{'<=': -51, 'F2': None, '>': -51, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -51, '=': None, '$': -51, ')': -51, ',': -51, 'S': None, '/': -51, '+': -51, ']': -51, 'then': -51, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -51, '<': -51, 'F': None, 'if': None, 'T': None, 'and': -51, '**': -51, 'lambda': None, '-': -51, '>=': -51, 'in': -51, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -51, '!=': -51, '*': -51, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -51, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -51, 'not': None},
  22:{'<=': -50, 'F2': None, '>': -50, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -50, '=': None, '$': -50, ')': -50, ',': -50, 'S': None, '/': -50, '+': -50, ']': -50, 'then': -50, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -50, '<': -50, 'F': None, 'if': None, 'T': None, 'and': -50, '**': -50, 'lambda': None, '-': -50, '>=': -50, 'in': -50, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -50, '!=': -50, '*': -50, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -50, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -50, 'not': None},
  23:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -2, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  24:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -16, ')': -16, ',': -16, 'S': None, '/': None, '+': None, ']': -16, 'then': -16, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -16, '<': None, 'F': None, 'if': None, 'T': None, 'and': 64, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -16, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -16, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  25:{'<=': -52, 'F2': None, '>': -52, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -52, '=': None, '$': -52, ')': -52, ',': -52, 'S': None, '/': -52, '+': -52, ']': -52, 'then': -52, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -52, '<': -52, 'F': None, 'if': None, 'T': None, 'and': -52, '**': -52, 'lambda': None, '-': -52, '>=': -52, 'in': -52, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -52, '!=': -52, '*': -52, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -52, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -52, 'not': None},
  26:{'<=': -45, 'F2': None, '>': -45, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -45, '=': None, '$': -45, ')': -45, ',': -45, 'S': None, '/': -45, '+': -45, ']': -45, 'then': -45, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -45, '<': -45, 'F': None, 'if': None, 'T': None, 'and': -45, '**': -45, 'lambda': None, '-': -45, '>=': -45, 'in': -45, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -45, '!=': -45, '*': -45, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -45, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -45, 'not': None},
  27:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -1, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  28:{'<=': -47, 'F2': None, '>': -47, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -47, '=': None, '$': -47, ')': -47, ',': -47, 'S': None, '/': -47, '+': -47, ']': -47, 'then': -47, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -47, '<': -47, 'F': None, 'if': None, 'T': None, 'and': -47, '**': -47, 'lambda': None, '-': -47, '>=': -47, 'in': -47, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -47, '!=': -47, '*': -47, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -47, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -47, 'not': None},
  29:{'<=': -49, 'F2': None, '>': -49, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -49, '=': None, '$': -49, ')': -49, ',': -49, 'S': None, '/': -49, '+': -49, ']': -49, 'then': -49, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -49, '<': -49, 'F': None, 'if': None, 'T': None, 'and': -49, '**': -49, 'lambda': None, '-': -49, '>=': -49, 'in': -49, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -49, '!=': -49, '*': -49, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -49, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -49, 'not': None},
  30:{'<=': -48, 'F2': None, '>': -48, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -48, '=': None, '$': -48, ')': -48, ',': -48, 'S': None, '/': -48, '+': -48, ']': -48, 'then': -48, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -48, '<': -48, 'F': None, 'if': None, 'T': None, 'and': -48, '**': -48, 'lambda': None, '-': -48, '>=': -48, 'in': -48, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -48, '!=': -48, '*': -48, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -48, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -48, 'not': None},
  31:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 65, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  32:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': 66, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 67, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  33:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 68, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  34:{'<=': None, 'F2': 42, '>': None, 'E': 69, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  35:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 70, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  36:{'<=': None, 'F2': 42, '>': None, 'E': 71, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  37:{'<=': None, 'F2': 42, '>': None, 'E': 72, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  38:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 73, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  39:{'<=': None, 'F2': 42, '>': None, 'E': 74, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  40:{'<=': None, 'F2': 42, '>': None, 'E': 75, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  41:{'<=': None, 'F2': 42, '>': None, 'E': 76, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  42:{'<=': -38, 'F2': None, '>': -38, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': 77, '=': None, '$': -38, ')': -38, ',': -38, 'S': None, '/': -38, '+': -38, ']': -38, 'then': -38, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -38, '<': -38, 'F': None, 'if': None, 'T': None, 'and': -38, '**': 33, 'lambda': None, '-': -38, '>=': -38, 'in': -38, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -38, '!=': -38, '*': -38, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -38, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -38, 'not': None},
  43:{'<=': 34, 'F2': None, '>': 36, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -27, ')': -27, ',': -27, 'S': None, '/': None, '+': 35, ']': -27, 'then': -27, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -27, '<': 41, 'F': None, 'if': None, 'T': None, 'and': -27, '**': None, 'lambda': None, '-': 38, '>=': 39, 'in': -27, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -27, '!=': 37, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': 40, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  44:{'<=': -30, 'F2': None, '>': -30, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -30, ')': -30, ',': -30, 'S': None, '/': 52, '+': -30, ']': -30, 'then': -30, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -30, '<': -30, 'F': None, 'if': None, 'T': None, 'and': -30, '**': None, 'lambda': None, '-': -30, '>=': -30, 'in': -30, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -30, '!=': -30, '*': 53, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -30, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': 54, 'not': None},
  45:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 55, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 78, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  46:{'<=': -43, 'F2': None, '>': -43, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -43, '=': None, '$': -43, ')': -43, ',': -43, 'S': None, '/': -43, '+': -43, ']': -43, 'then': -43, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -43, '<': -43, 'F': None, 'if': None, 'T': None, 'and': -43, '**': -43, 'lambda': None, '-': -43, '>=': -43, 'in': -43, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -43, '!=': -43, '*': -43, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -43, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -43, 'not': None},
  47:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 79, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  48:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -16, ')': -16, ',': -16, 'S': None, '/': None, '+': None, ']': -16, 'then': -16, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -16, '<': None, 'F': None, 'if': None, 'T': None, 'and': 64, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -16, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -16, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  49:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': 80, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  50:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': 58, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  51:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': 81, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': 63, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  52:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 82, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  53:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 83, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  54:{'<=': None, 'F2': 42, '>': None, 'E': None, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': 6, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': 84, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': None},
  55:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 85, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  56:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': 86, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 87, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  57:{'<=': -35, 'F2': None, '>': -35, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -35, ')': -35, ',': -35, 'S': None, '/': -35, '+': -35, ']': -35, 'then': -35, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -35, '<': -35, 'F': None, 'if': None, 'T': None, 'and': -35, '**': None, 'lambda': None, '-': -35, '>=': -35, 'in': -35, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -35, '!=': -35, '*': -35, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -35, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -35, 'not': None},
  58:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 88, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  59:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': 89, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  60:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': 90, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  61:{'<=': -55, 'F2': None, '>': -55, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -55, '=': None, '$': -55, ')': -55, ',': -55, 'S': None, '/': -55, '+': -55, ']': -55, 'then': -55, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -55, '<': -55, 'F': None, 'if': None, 'T': None, 'and': -55, '**': -55, 'lambda': None, '-': -55, '>=': -55, 'in': -55, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -55, '!=': -55, '*': -55, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -55, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -55, 'not': None},
  62:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': 91, 'S': None, '/': None, '+': None, ']': -58, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  63:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': 92, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  64:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 93, 'let': None, ':': None, 'or': None, '<': None, 'F': 10, 'if': None, 'T': 44, 'and': None, '**': None, 'lambda': None, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': None, 'Band': None, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  65:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -19, ')': -19, ',': -19, 'S': None, '/': None, '+': None, ']': -19, 'then': -19, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -19, '<': None, 'F': None, 'if': None, 'T': None, 'and': -19, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -19, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -19, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  66:{'<=': -39, 'F2': None, '>': -39, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -39, '=': None, '$': -39, ')': -39, ',': -39, 'S': None, '/': -39, '+': -39, ']': -39, 'then': -39, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -39, '<': -39, 'F': None, 'if': None, 'T': None, 'and': -39, '**': -39, 'lambda': None, '-': -39, '>=': -39, 'in': -39, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -39, '!=': -39, '*': -39, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -39, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -39, 'not': None},
  67:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 94, ',': 95, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  68:{'<=': -37, 'F2': None, '>': -37, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -37, ')': -37, ',': -37, 'S': None, '/': -37, '+': -37, ']': -37, 'then': -37, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -37, '<': -37, 'F': None, 'if': None, 'T': None, 'and': -37, '**': None, 'lambda': None, '-': -37, '>=': -37, 'in': -37, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -37, '!=': -37, '*': -37, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -37, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -37, 'not': None},
  69:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -24, ')': -24, ',': -24, 'S': None, '/': None, '+': 35, ']': -24, 'then': -24, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -24, '<': None, 'F': None, 'if': None, 'T': None, 'and': -24, '**': None, 'lambda': None, '-': 38, '>=': None, 'in': -24, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -24, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  70:{'<=': -28, 'F2': None, '>': -28, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -28, ')': -28, ',': -28, 'S': None, '/': 52, '+': -28, ']': -28, 'then': -28, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -28, '<': -28, 'F': None, 'if': None, 'T': None, 'and': -28, '**': None, 'lambda': None, '-': -28, '>=': -28, 'in': -28, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -28, '!=': -28, '*': 53, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -28, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': 54, 'not': None},
  71:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -25, ')': -25, ',': -25, 'S': None, '/': None, '+': 35, ']': -25, 'then': -25, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -25, '<': None, 'F': None, 'if': None, 'T': None, 'and': -25, '**': None, 'lambda': None, '-': 38, '>=': None, 'in': -25, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -25, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  72:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -22, ')': -22, ',': -22, 'S': None, '/': None, '+': 35, ']': -22, 'then': -22, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -22, '<': None, 'F': None, 'if': None, 'T': None, 'and': -22, '**': None, 'lambda': None, '-': 38, '>=': None, 'in': -22, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -22, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  73:{'<=': -29, 'F2': None, '>': -29, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -29, ')': -29, ',': -29, 'S': None, '/': 52, '+': -29, ']': -29, 'then': -29, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -29, '<': -29, 'F': None, 'if': None, 'T': None, 'and': -29, '**': None, 'lambda': None, '-': -29, '>=': -29, 'in': -29, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -29, '!=': -29, '*': 53, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -29, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': 54, 'not': None},
  74:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -26, ')': -26, ',': -26, 'S': None, '/': None, '+': 35, ']': -26, 'then': -26, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -26, '<': None, 'F': None, 'if': None, 'T': None, 'and': -26, '**': None, 'lambda': None, '-': 38, '>=': None, 'in': -26, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -26, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  75:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -21, ')': -21, ',': -21, 'S': None, '/': None, '+': 35, ']': -21, 'then': -21, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -21, '<': None, 'F': None, 'if': None, 'T': None, 'and': -21, '**': None, 'lambda': None, '-': 38, '>=': None, 'in': -21, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -21, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  76:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -23, ')': -23, ',': -23, 'S': None, '/': None, '+': 35, ']': -23, 'then': -23, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -23, '<': None, 'F': None, 'if': None, 'T': None, 'and': -23, '**': None, 'lambda': None, '-': 38, '>=': None, 'in': -23, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -23, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  77:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': 66, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 96, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  78:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': 86, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 87, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  79:{'<=': -54, 'F2': None, '>': -54, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -54, '=': None, '$': -54, ')': -54, ',': -54, 'S': None, '/': -54, '+': -54, ']': -54, 'then': -54, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -54, '<': -54, 'F': None, 'if': None, 'T': None, 'and': -54, '**': -54, 'lambda': None, '-': -54, '>=': -54, 'in': -54, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -54, '!=': -54, '*': -54, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -54, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -54, 'not': None},
  80:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 97, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  81:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 98, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  82:{'<=': -32, 'F2': None, '>': -32, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -32, ')': -32, ',': -32, 'S': None, '/': -32, '+': -32, ']': -32, 'then': -32, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -32, '<': -32, 'F': None, 'if': None, 'T': None, 'and': -32, '**': None, 'lambda': None, '-': -32, '>=': -32, 'in': -32, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -32, '!=': -32, '*': -32, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -32, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -32, 'not': None},
  83:{'<=': -31, 'F2': None, '>': -31, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -31, ')': -31, ',': -31, 'S': None, '/': -31, '+': -31, ']': -31, 'then': -31, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -31, '<': -31, 'F': None, 'if': None, 'T': None, 'and': -31, '**': None, 'lambda': None, '-': -31, '>=': -31, 'in': -31, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -31, '!=': -31, '*': -31, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -31, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -31, 'not': None},
  84:{'<=': -33, 'F2': None, '>': -33, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -33, ')': -33, ',': -33, 'S': None, '/': -33, '+': -33, ']': -33, 'then': -33, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -33, '<': -33, 'F': None, 'if': None, 'T': None, 'and': -33, '**': None, 'lambda': None, '-': -33, '>=': -33, 'in': -33, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -33, '!=': -33, '*': -33, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -33, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -33, 'not': None},
  85:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -9, ')': -9, ',': -9, 'S': None, '/': None, '+': None, ']': -9, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -9, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -9, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  86:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 99, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  87:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 100, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  88:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -7, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -7, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  89:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 101, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 102, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  90:{'<=': -56, 'F2': None, '>': -56, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -56, '=': None, '$': -56, ')': -56, ',': -56, 'S': None, '/': -56, '+': -56, ']': -56, 'then': -56, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -56, '<': -56, 'F': None, 'if': None, 'T': None, 'and': -56, '**': -56, 'lambda': None, '-': -56, '>=': -56, 'in': -56, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -56, '!=': -56, '*': -56, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -56, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -56, 'not': None},
  91:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': 103, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 62, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  92:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -15, ')': -15, ',': -15, 'S': None, '/': None, '+': None, ']': -15, 'then': -15, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -15, '<': None, 'F': None, 'if': None, 'T': None, 'and': 64, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -15, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -15, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  93:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -17, ')': -17, ',': -17, 'S': None, '/': None, '+': None, ']': -17, 'then': -17, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -17, '<': None, 'F': None, 'if': None, 'T': None, 'and': -17, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -17, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -17, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  94:{'<=': -40, 'F2': None, '>': -40, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -40, '=': None, '$': -40, ')': -40, ',': -40, 'S': None, '/': -40, '+': -40, ']': -40, 'then': -40, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -40, '<': -40, 'F': None, 'if': None, 'T': None, 'and': -40, '**': -40, 'lambda': None, '-': -40, '>=': -40, 'in': -40, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -40, '!=': -40, '*': -40, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -40, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -40, 'not': None},
  95:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 104, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  96:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 94, ',': 105, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  97:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -13, ')': -13, ',': -13, 'S': None, '/': None, '+': None, ']': -13, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -13, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -13, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  98:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': 106, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  99:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': 108, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 107, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  100:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -10, ')': -10, ',': -10, 'S': None, '/': None, '+': None, ']': -10, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -10, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -10, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  101:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 109, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  102:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 110, ',': 111, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  103:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': -57, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  104:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 112, ',': 113, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  105:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 114, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  106:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 115, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  107:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 116, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  108:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 117, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  109:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': 118, 'None': None, 'False': None, '%': None, 'not': None},
  110:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 119, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  111:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 120, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  112:{'<=': -41, 'F2': None, '>': -41, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -41, '=': None, '$': -41, ')': -41, ',': -41, 'S': None, '/': -41, '+': -41, ']': -41, 'then': -41, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -41, '<': -41, 'F': None, 'if': None, 'T': None, 'and': -41, '**': -41, 'lambda': None, '-': -41, '>=': -41, 'in': -41, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -41, '!=': -41, '*': -41, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -41, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -41, 'not': None},
  113:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 121, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  114:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 112, ',': 113, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  115:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -8, ')': -8, ',': -8, 'S': None, '/': None, '+': None, ']': -8, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -8, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -8, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  116:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -11, ')': -11, ',': -11, 'S': None, '/': None, '+': None, ']': -11, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -11, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -11, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  117:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 122, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  118:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 123, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  119:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': 124, 'None': None, 'False': None, '%': None, 'not': None},
  120:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 125, ',': 126, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  121:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 127, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  122:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 128, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  123:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -3, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  124:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 129, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  125:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 130, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  126:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': 131, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  127:{'<=': -42, 'F2': None, '>': -42, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': -42, '=': None, '$': -42, ')': -42, ',': -42, 'S': None, '/': -42, '+': -42, ']': -42, 'then': -42, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': -42, '<': -42, 'F': None, 'if': None, 'T': None, 'and': -42, '**': -42, 'lambda': None, '-': -42, '>=': -42, 'in': -42, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -42, '!=': -42, '*': -42, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': -42, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': -42, 'not': None},
  128:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -12, ')': -12, ',': -12, 'S': None, '/': None, '+': None, ']': -12, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': -12, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': -12, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  129:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -4, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  130:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': 132, 'None': None, 'False': None, '%': None, 'not': None},
  131:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': 133, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  132:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 134, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  133:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': 135, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  134:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -5, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None},
  135:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': 136, 'None': None, 'False': None, '%': None, 'not': None},
  136:{'<=': None, 'F2': 42, '>': None, 'E': 43, 'rest': 3, 'float': 4, 'EList': None, '(': 5, '=': None, '$': None, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': 7, 'F1': 6, 'Bnot': 8, 'let': 9, ':': None, 'or': None, '<': None, 'F': 10, 'if': 11, 'T': 44, 'and': None, '**': None, 'lambda': 45, '-': 14, '>=': None, 'in': None, 'F3': 15, 'Top': None, 'id': 46, 'def': None, '[': 19, 'Bor': 20, 'else': None, '!=': None, '*': None, 'cons': 21, 'nil': 22, 'Expr': 137, 'Band': 48, 'int': 26, '==': None, 'first': 25, 'Assign': None, 'True': 28, 'return': None, 'None': 29, 'False': 30, '%': None, 'not': 31},
  137:{'<=': None, 'F2': None, '>': None, 'E': None, 'rest': None, 'float': None, 'EList': None, '(': None, '=': None, '$': -6, ')': None, ',': None, 'S': None, '/': None, '+': None, ']': None, 'then': None, 'C': None, 'F1': None, 'Bnot': None, 'let': None, ':': None, 'or': None, '<': None, 'F': None, 'if': None, 'T': None, 'and': None, '**': None, 'lambda': None, '-': None, '>=': None, 'in': None, 'F3': None, 'Top': None, 'id': None, 'def': None, '[': None, 'Bor': None, 'else': None, '!=': None, '*': None, 'cons': None, 'nil': None, 'Expr': None, 'Band': None, 'int': None, '==': None, 'first': None, 'Assign': None, 'True': None, 'return': None, 'None': None, 'False': None, '%': None, 'not': None}
}



def action(lst,table):
    state = 0
    for token in lst:      
        try:
            state = table[state][token[0]]
        except KeyError:
            return err          
        if state == err:        # error state is implicit so fail as soon as you hit an error
            return err
    return state



### Reminder: You should not modify anything above this line unless you check with Professor Snyder first!
<pre>




</pre>

## Problem One: MyPython 1.2 ASTs  (10 pts)

As mentioned in lecture on Thursday, the task for this week is to add to MyPython:
<ul>
    <li> Function and lambda definitions with variable numbers of parameters; we will do 0 -- 3 parameters;
    <li> Closures to deal with function definitions;
    <li> The list data type (following Lisp).
</ul>
The new grammar is below, 59 rules! (The parser DFA has 138 states.) The parser code below is my solution
with certain cases left out (see the places where I say "Your code here"), and with rules
from last week renumbered.  Your task, as usual, is to adapt this to the new features. You should do this in
three stages, confirming by running the appropriate tests that your code works as expected. 

### Part A: Variable parameter and argument lists
Tuesday's lecture outlined the motivation and some of the details.  The basic idea is that
in place of the bound variable in the AST for a lambda expression, we will have a 'params' tuple which
contains 0 -- 3 variables, i.e., instead of something like this:
<pre>
            ('lambda', ('id', 'x'),  &lt; body &gt; )
</pre>
you will have something like this (of course the variable names may be different):
<pre>
            ('lambda', ('params',  ),  &lt; body &gt; )
            ('lambda', ('params', ('id', 'x') ),  &lt; body &gt; )
            ('lambda', ('params', ('id', 'x'), ('id', 'y') ),  &lt; body &gt; )
            ('lambda', ('params', ('id', 'x'), ('id', 'y'), ('id', 'z') ),  &lt; body &gt; )
</pre>
In the grammar these will be created from rules 9 -- 12 and assignment
statements using these new ASTs will be created from 3 - 6. 
<p>
To account for the actual arguments in an application, you will need to create corresponding
'args" tuples for rules 39 -- 42. Instead of 
<pre>
f(x)        represented as      ('apply',('id', 'f'),('id', 'x'))
</pre>
we will now have ASTs in the general form:
<pre>
f()         represented as      ('apply', ('id', 'f'), ('args',) )
f(5)        represented as      ('apply', ('id', 'f'), ('args',('int', 5)) )
f(x,9)      represented as      ('apply', ('id', 'f'), ('args',('id', 'x'),('int', 9)) )
f(1,8,z)    represented as      ('apply', ('id', 'f'), ('args',('int', 1),('int', 8),('id', 'x')) )
</pre>


### Part B: Closures
The second part of the video lecture for Lecture 8 contains all the background material you need to do this part, and really all you need to do is to change the ASTS for lambda expressions so that the last component is an empty list for the closure environment, so instead of 
 <pre>
                  ('lambda', ('id', 'x'),  &lt; body &gt; )
</pre>
you will now, in addition to wrapping the bound variable in a params list, have a 4th component:
<pre>
                  ('lambda', ('params', ('id', 'x') ),  &lt; body &gt; , [] )
</pre>

### Part C: List Data Type
For this part, you will need to add code for rules 55 - 58 to "cons-up" lists; these will be written in
MyPython as normal Python lists of expressions, but represented in the AST with 'cons' and 'nil' tuples; when pretty-printed they will be turned back into the familiar notation, but the important thing is that you need to "store"
the ASTs for the expressions in the list inside the AST. 

Here are some example lists:
<pre>
[]        will be represented as    ('nil',)
nil       will be represented as    ('nil',)
[1,x]     will be represented as    ('cons', ('int',1), ('cons', ('id','x'), ('nil',) )
[x+1,y]   will be represented as    ('cons', ('+', ('id', 'x'), ('int', 1)), ('cons', ('id', 'y'), ('nil',)))
[1,[y],3] will be represented as         
                ('cons', ('int', 1), ('cons', ('cons', ('id', 'y'), ('nil',)), ('cons', ('int', 3), ('nil',))))
</pre>
The important thing to notice is that the grammar allows arbitrary expressions as elements of
lists (which is how we get nested lists), so you just have to put the ASTs for each Expr in the appropriate place.
Note that rule 57 has been written "right-recursively" so as to help you create the appropriate
nested structure for the cons tuples. 

The test cases should answer any other questions. 

In [4]:
print("\nGrammar for MyPython 1.2:\n")
pprint_grammar(Grammar1_2)


Grammar for MyPython 1.2:

0: S := Top 
1: Top := Assign 
2: Top := Expr 
3: Top := def id ( ) : return Expr 
4: Top := def id ( id ) : return Expr 
5: Top := def id ( id , id ) : return Expr 
6: Top := def id ( id , id , id ) : return Expr 
7: Assign := id = Expr 
8: Expr := if Bor then Expr else Expr 
9: Expr := lambda : Expr 
10: Expr := lambda id : Expr 
11: Expr := lambda id , id : Expr 
12: Expr := lambda id , id , id : Expr 
13: Expr := let Assign in Expr 
14: Expr := Bor 
15: Bor := Bor or Band 
16: Bor := Band 
17: Band := Band and Bnot 
18: Band := Bnot 
19: Bnot := not Bnot 
20: Bnot := C 
21: C := E == E 
22: C := E != E 
23: C := E < E 
24: C := E <= E 
25: C := E > E 
26: C := E >= E 
27: C := E 
28: E := E + T 
29: E := E - T 
30: E := T 
31: T := T * F 
32: T := T / F 
33: T := T % F 
34: T := F 
35: F := - F 
36: F := F1 
37: F1 := F2 ** F1 
38: F1 := F2 
39: F2 := F2 ( ) 
40: F2 := F2 ( Expr ) 
41: F2 := F2 ( Expr , Expr ) 
42: F2 := F2 ( Expr , Expr , Expr ) 
43: F2

In [5]:
# Code for shift-reduce parser

# pretty-printing parser configurations

def pprint_parser(parsingStack,inputListOfTokens):
    totalWidth = 80
    smallestGap = 6
    largestStackLength = int(totalWidth*0.6 - smallestGap/2)   # most characters to display
    largestInputLength = totalWidth - largestStackLength - smallestGap
    
    p = '| '
    for symbol in parsingStack:
        p = p + tokenToString(symbol) + ' '
    if len(p) > largestStackLength:
        ind = int(len(p) - largestStackLength + 9)
        p = '| ... ' + p[ind:]
        
    q = ""
    for tok in inputListOfTokens[:-1]:
        q = q + tokenToString(tok) + ' '
    if len(inputListOfTokens) > 0:
        q = q + tokenToString(inputListOfTokens[-1]) 

    if len(q) > largestInputLength:
        ind = int(largestInputLength - 9)
        q = q[:ind] + ' ... ' 
    
    q = q + ' |'
        
    p = p + (' ' * (totalWidth - len(p) - len(q)))
    print(p+q)
    
# Pretty-printing code for testing your solution

def AST_to_string(t):
    if t == None:
        return None
    elif t == []:
        return '[]'
    elif type(t) == list:
        s = '['
        for el in t[:-1]:
            s += AST_to_string(el) + ','
        s += AST_to_string(t[-1])
        return s + ']'
    elif type(t) != tuple:
        return str(t)
    elif t[0] in ['int','id','float']:
        return str(t[1])
    elif t[0] == nil:
        return 'nil'
    elif len(t) == 1 and (t[0] == 'params' or t[0] == 'args'):
        return '()'
    elif len(t) == 1 and t[0] in ['cons','first','rest']:
        return t[0]
    elif len(t) == 1:
        return '(' + AST_to_string(t[0]) + ')'
    elif len(t) == 2 and t[0] in ['-', 'not']:
        return t[0] + '(' + AST_to_string(t[1]) + ')'
    elif len(t) == 2 and (t[0] == 'params' or t[0] == 'args'):
        return '(' + AST_to_string(t[1]) + ')'
    elif len(t) == 2:
        return '(' + AST_to_string(t[0]) + ',' + AST_to_string(t[1]) + ')'
    elif len(t) == 3 and t[0] in ['*','+', '/', '%', '**', '-','and', 'or','<','>','!=','==','>=','<=']:
        return  '(' + AST_to_string(t[1]) + ' ' + t[0] + ' ' + AST_to_string(t[2]) + ')'
    elif len(t) == 3 and t[0] == 'apply':
        s = AST_to_string(t[1])
        if t[2] == None:
            return s + '()'
        else:
            return s + AST_to_string(t[2]) 
    elif len(t) == 3 and t[0] == '=':
        return  AST_to_string(t[1]) + ' = ' + AST_to_string(t[2])
    elif len(t) == 3 and t[0] == 'let':
        return '(' + listAST_to_string(t[0]) + ' ' + AST_to_string(t[1]) + ' in ' + AST_to_string(t[2]) + ')'
    elif len(t) == 3 and t[0] == 'cons':
        return listAST_to_string(t)
    elif len(t) == 3 and (t[0] == 'params' or t[0] == 'args'):
        return '(' + AST_to_string(t[1]) + ',' + AST_to_string(t[2]) + ')'
    elif len(t) == 3:
        return '(' + AST_to_string(t[0]) + ' ' + AST_to_string(t[1]) + ', ' + AST_to_string(t[2]) + ')'
    elif len(t) == 4 and t[0] == 'if':
        return '(' + t[0] + ' ' + AST_to_string(t[1]) + ' then ' + AST_to_string(t[2]) + ' else ' + AST_to_string(t[3]) + ')'
    elif len(t) == 4 and (t[0] == 'params' or t[0] == 'args'):
        return '(' + AST_to_string(t[1]) + ',' + AST_to_string(t[2]) + ',' + AST_to_string(t[3]) + ')'
    elif len(t) == 4 and t[0] == 'lambda':
        return "(\u03bb" +  AST_to_string(t[1]) + '. ' + AST_to_string(t[2]) + ' @ ' + str(t[3]) + ')'
    #else
    print("** " + str(len(t)))
    print("** " + str(t))
    return "ERROR!"


    
def listAST_to_string(L):
    if type(L) != tuple:
        return str(L)
    elif L[0] != 'cons':
        return AST_to_string(L)
    else:
        return "[" + lts_helper(L)

def lts_helper(L):
    if L == nil or L == ('nil',):
        return "]"
    elif L[2] == (nil,):     # list with one element
        return listAST_to_string(L[1]) + ']'
    else:
        return listAST_to_string(L[1]) + ',' + lts_helper(L[2])
    
def pprint_list(L):
    print(list_to_string(L))

def pprint_AST(t):
    pprint_AST_helper(t,'')

def pprint_AST_helper(t,indent):
    if t == None:
        print(indent+'()')
    elif type(t) == list:
        print(indent + str(t))
    elif (t[0] in [ident,integer,floating]):
        print(indent + '(' + t[0] + ',' + str(t[1]) + ')')
    elif (t[0] in ['True', 'False', 'nil']):
        print(indent + '(' + t[0] + ',)')
    else:
        print(indent + t[0])
        for k in range(1,len(t)):
            pprint_AST_helper(t[k],indent + '\t')    

# parse takes a list of tokens and determines if it is a 
# well-formed arithmetic expression.
# This version will also build an AST

def parse(list_of_input_tokens,rules,table,verbose=False):
    
    # add end of input marker
    list_of_input_tokens.append( (end_of_input,) )
    
    # input stack; use pop(0) to remove front item in list
    input_stack = list_of_input_tokens
    
    # parsing stack; use append to push onto end and pop() to pop from end
    parsing_stack = []
    
    if verbose:
        print('Input Tokens: ' + tokenListToString(input_stack) + '\n')
        pprint_parser(parsing_stack,input_stack) 
    
    while( len(input_stack) > 0 ):
        
        # Now we will "lookahead" at the next symbol on the input and ask the automaton what to do
        # a positive number means to shift, and a negative number -n means to reduce by rule n
        
        n = action(parsing_stack+[input_stack[0]],table)
        if n == accept:   # reduce by start rule, success
            if verbose:
                print("\nAccept!")
            return parsing_stack[-1][1] 
        elif n == err:     #  problem on stack!
            if verbose:
                print("\nERROR: No transition here:")
                pprint_parser(parsing_stack,input_stack)
            return None

        elif n > 0:     # shift 
            if verbose:
                print('\naction: ' + str(n) + '\tshift\n')
            parsing_stack.append( input_stack.pop(0)) 
        else:         # reduce by rule -n 
            if verbose:
                print('\naction: ' + str(n) + '\treduce by rule ' + str(-n) + ': ' + rule_to_string(rules[-n])+'\n')
            r = -n
            LHS = rules[r][0]
            if r in [0,1,2,14,16,18,20,27,30,34,36,38,44]:  # unary rules just pass ast up 
                (_,ast) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],ast) )
                
            # Your code here for rules 3 - 6
            elif r == 3:
                (_,ast) = parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop() 
                parsing_stack.pop() 
                t =parsing_stack.pop() 
                parsing_stack.pop() 
                parsing_stack.append((LHS[0], (assign,t,(lambdaTok,('params',),ast,[]))))
            elif r == 4:
                (_,ast) = parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                t =parsing_stack.pop() 
                parsing_stack.pop()  
                t1 = parsing_stack.pop() 
                parsing_stack.pop() 
                parsing_stack.append((LHS[0], (assign,t1,(lambdaTok,('params',t), ast,[]))))
            elif r == 5:
                (_,ast) = parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                t =parsing_stack.pop() 
                parsing_stack.pop()  
                tt =parsing_stack.pop()
                parsing_stack.pop() 
                t1 = parsing_stack.pop() 
                parsing_stack.pop() 
                parsing_stack.append((LHS[0], (assign,t1,(lambdaTok,('params',tt,t), ast,[]))))
            elif r in [6]:
                (_,ast) = parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                parsing_stack.pop()  
                t =parsing_stack.pop() 
                parsing_stack.pop()  
                tt =parsing_stack.pop()
                parsing_stack.pop() 
                ttt =parsing_stack.pop()
                parsing_stack.pop()
                t1 = parsing_stack.pop() 
                parsing_stack.pop() 
                parsing_stack.append((LHS[0], (assign,t1,(lambdaTok,('params',ttt,tt,t), ast,[]))))
            
            elif r == 7:                           # Assign := id = Expr
                (_,ast) = parsing_stack.pop()      # Expr
                parsing_stack.pop()                # =
                t = parsing_stack.pop()            # id               
                parsing_stack.append( (LHS[0],(assign,t,ast) ) )
            elif r == 8:                           # Expr := if Bor then Expr else Expr
                (_,ast3) = parsing_stack.pop()     # Expr 
                parsing_stack.pop()                # else
                (_,ast2) = parsing_stack.pop()     # Expr  
                parsing_stack.pop()                # then
                (_,ast1) = parsing_stack.pop()     # Bor
                (operator,) = parsing_stack.pop()  # if
                parsing_stack.append( (LHS[0],(ifTok,ast1,ast2,ast3) ) )
                
            # Your code here for rules 9 -- 11
            elif r ==9:
                (_,ast2) = parsing_stack.pop()
                parsing_stack.pop()
                (lam,) = parsing_stack.pop()
                parsing_stack.append((LHS[0], (lam,('params',),ast2,[]))) 
            elif r ==10:
                (_,ast2) = parsing_stack.pop()
                parsing_stack.pop()
                id1 = parsing_stack.pop()
                (lam,) = parsing_stack.pop()
                parsing_stack.append((LHS[0], (lam,('params',id1),ast2,[]))) 
            elif r ==11:
                (_,ast2) = parsing_stack.pop()
                parsing_stack.pop()
                id1 = parsing_stack.pop()
                parsing_stack.pop()
                id2 = parsing_stack.pop()
                (lam,) = parsing_stack.pop()
                parsing_stack.append((LHS[0], (lam,('params',id2,id1),ast2,[]))) 
            elif r in[12]:
                (_,ast2) = parsing_stack.pop()
                parsing_stack.pop()
                id1 = parsing_stack.pop()
                parsing_stack.pop()
                id2 = parsing_stack.pop()
                parsing_stack.pop()
                id3 = parsing_stack.pop()
                (lam,) = parsing_stack.pop()
                parsing_stack.append((LHS[0], (lam,('params',id3,id2,id1),ast2,[]))) 
            elif r == 13:                          # Expr := let Assign in Expr  
                (_,ast2) = parsing_stack.pop()     # Expr
                parsing_stack.pop()                # in 
                (_,ast1) = parsing_stack.pop()     # Assign
                parsing_stack.pop()                # let 
                parsing_stack.append( (LHS[0],(let,ast1,ast2)) )
            elif r in [15,17,21,22,23,24,25,26,28,29,31,32,33,37]:   # binary operators
                (_,ast2) = parsing_stack.pop()
                (operator,) = parsing_stack.pop()
                (_,ast1) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(operator,ast1,ast2)) )
            elif r in [19,35]:                     # unary operators
                (_,ast) = parsing_stack.pop()      # F  | Bnot
                (operator,) = parsing_stack.pop()  # - | not
                parsing_stack.append( (LHS[0],(operator,ast)) )
            
            # Your code here for rules 39 - 42
            elif r == 39:
                parsing_stack.pop() 
                parsing_stack.pop() 
                (_,ast) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(apply,ast,('args',))) )
            elif r in [40]:
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                (_,ast2) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(apply,ast2,('args',ast))) )
            elif r in[41]:
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                (_,ast1) = parsing_stack.pop()      # E
                parsing_stack.pop() 
                (_,ast2) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(apply,ast2,('args',ast1,ast)) ))
            elif r in [42]:
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                (_,ast1) = parsing_stack.pop()      # E
                parsing_stack.pop() 
                (_,ast11) =parsing_stack.pop() 
                parsing_stack.pop() 
                (_,ast2) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],(apply,ast2,('args',ast11,ast1,ast))) )
            # Hm... I wonder where rules 51 - 53 should go??
            
            elif r in [43,45,46,47,48,49,50,51,52,53]:      # rules for values create leaf node by passing tuple up
                t = parsing_stack.pop()            
                parsing_stack.append( (LHS[0],t) )
            elif r in [54]:                          # F3 := ( Expr )
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                parsing_stack.append( (LHS[0],ast) )
            
            # Your code for rules 45 - 58
            elif r == 55:
                parsing_stack.pop()                
                parsing_stack.pop()                
                parsing_stack.append( (LHS[0],('nil',)) )
            elif r in [56]:
                parsing_stack.pop()                # (
                (_,ast) = parsing_stack.pop()      # E
                parsing_stack.pop()                # )
                parsing_stack.append( (LHS[0],ast))
            elif r == 57:
                (_,ast2) = parsing_stack.pop()
                parsing_stack.pop()
                (_,ast1) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],('cons',ast1,ast2) ))
            elif r == 58:
                (_,ast1) = parsing_stack.pop()
                parsing_stack.append( (LHS[0],('cons',ast1,('nil',)) ))
            else:
                print("parse stack error: action " + str(n))
                return None
   
        if verbose:
            pprint_parser(parsing_stack,input_stack)
            
    return None     # this will never be executed

def testParserAST(s,verbose=False):
    t = parse(lexer(s),Grammar1_1,table,verbose)
    if t == None or t == False:
        print("Error: No AST generated!")
    else:
        print('Input: ' + str(s))
        print("\nAbstract Syntax Tree:\n")
        print(AST_to_string(t)) 
        print()
        pprintAST(t)



In [6]:
# Test 1
s = 'f = lambda x,y : x + y'
t = 'def f(x,y) : return x + y'
a = parse(lexer(s),Grammar1_2,table)
b = parse(lexer(t),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print(AST_to_string(b))
print()
pprint_AST(a)

('=', ('id', 'f'), ('lambda', ('params', ('id', 'x'), ('id', 'y')), ('+', ('id', 'x'), ('id', 'y')), []))

f = (λ(x,y). (x + y) @ [])
f = (λ(x,y). (x + y) @ [])

=
	(id,f)
	lambda
		params
			(id,x)
			(id,y)
		+
			(id,x)
			(id,y)
		[]


In [7]:
# Test 2
s = 'f = lambda : 3 * 5'
t = 'def f() : return 3 * 5'
a = parse(lexer(s),Grammar1_2,table)
b = parse(lexer(t),Grammar1_2,table)
print(AST_to_string(a))
print(AST_to_string(b))
pprint_AST(a)

f = (λ(). (3 * 5) @ [])
f = (λ(). (3 * 5) @ [])
=
	(id,f)
	lambda
		params
		*
			(int,3)
			(int,5)
		[]


In [8]:
# Test 3
s = 'f = lambda x: lambda y,z : x + y * z'
t = 'def f(x) : return lambda y,z : x + y * z'
a = parse(lexer(s),Grammar1_2,table)
b = parse(lexer(t),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print(AST_to_string(b))
print()
pprint_AST(a)

('=', ('id', 'f'), ('lambda', ('params', ('id', 'x')), ('lambda', ('params', ('id', 'y'), ('id', 'z')), ('+', ('id', 'x'), ('*', ('id', 'y'), ('id', 'z'))), []), []))

f = (λ(x). (λ(y,z). (x + (y * z)) @ []) @ [])
f = (λ(x). (λ(y,z). (x + (y * z)) @ []) @ [])

=
	(id,f)
	lambda
		params
			(id,x)
		lambda
			params
				(id,y)
				(id,z)
			+
				(id,x)
				*
					(id,y)
					(id,z)
			[]
		[]


In [9]:
# Test 4
s = 'f(a,g(),h(3))'
a = parse(lexer(s),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

('apply', ('id', 'f'), ('args', ('id', 'a'), ('apply', ('id', 'g'), ('args',)), ('apply', ('id', 'h'), ('args', ('int', 3)))))

f(a,g(),h(3))

apply
	(id,f)
	args
		(id,a)
		apply
			(id,g)
			args
		apply
			(id,h)
			args
				(int,3)


In [10]:
# Test 5
s = '(lambda f,y,z : f(x,y))(2 * 4, g(9/flag),[a,s])'
a = parse(lexer(s),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

('apply', ('lambda', ('params', ('id', 'f'), ('id', 'y'), ('id', 'z')), ('apply', ('id', 'f'), ('args', ('id', 'x'), ('id', 'y'))), []), ('args', ('*', ('int', 2), ('int', 4)), ('apply', ('id', 'g'), ('args', ('/', ('int', 9), ('id', 'flag')))), ('cons', ('id', 'a'), ('cons', ('id', 's'), ('nil',)))))

(λ(f,y,z). f(x,y) @ [])((2 * 4),g((9 / flag)),[a,s])

apply
	lambda
		params
			(id,f)
			(id,y)
			(id,z)
		apply
			(id,f)
			args
				(id,x)
				(id,y)
		[]
	args
		*
			(int,2)
			(int,4)
		apply
			(id,g)
			args
				/
					(int,9)
					(id,flag)
		cons
			(id,a)
			cons
				(id,s)
				(nil,)


In [11]:
# Test 6
s = '[]'
a = parse(lexer(s),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

('nil',)

nil

(nil,)


In [12]:
# Test 7
s = '[3,5*x,f(x)]'
a = parse(lexer(s),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

('cons', ('int', 3), ('cons', ('*', ('int', 5), ('id', 'x')), ('cons', ('apply', ('id', 'f'), ('args', ('id', 'x'))), ('nil',))))

[3,(5 * x),f(x)]

cons
	(int,3)
	cons
		*
			(int,5)
			(id,x)
		cons
			apply
				(id,f)
				args
					(id,x)
			(nil,)


In [13]:
# Test 8
s = '[ 1, [2,3*2], [4,[5],[]],6]'
a = parse(lexer(s),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

('cons', ('int', 1), ('cons', ('cons', ('int', 2), ('cons', ('*', ('int', 3), ('int', 2)), ('nil',))), ('cons', ('cons', ('int', 4), ('cons', ('cons', ('int', 5), ('nil',)), ('cons', ('nil',), ('nil',)))), ('cons', ('int', 6), ('nil',)))))

[1,[2,(3 * 2)],[4,[5],nil],6]

cons
	(int,1)
	cons
		cons
			(int,2)
			cons
				*
					(int,3)
					(int,2)
				(nil,)
		cons
			cons
				(int,4)
				cons
					cons
						(int,5)
						(nil,)
					cons
						(nil,)
						(nil,)
			cons
				(int,6)
				(nil,)


In [14]:
# Test 9
s = 'cons(first(L), rest(L))'
print(lexer(s))
a = parse(lexer(s),Grammar1_2,table)

print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

[('cons',), ('(',), ('first',), ('(',), ('id', 'L'), (')',), (',',), ('rest',), ('(',), ('id', 'L'), (')',), (')',)]
('apply', ('cons',), ('args', ('apply', ('first',), ('args', ('id', 'L'))), ('apply', ('rest',), ('args', ('id', 'L')))))

cons(first(L),rest(L))

apply
	cons
	args
		apply
			first
			args
				(id,L)
		apply
			rest
			args
				(id,L)


In [15]:
# Test 10
s = 'def max(x,y,z) : return if  x > y then if z > x then z else x else if z > y then z else y'
a = parse(lexer(s),Grammar1_2,table)
print(a)
print()
print(AST_to_string(a))
print()
pprint_AST(a)

('=', ('id', 'max'), ('lambda', ('params', ('id', 'x'), ('id', 'y'), ('id', 'z')), ('if', ('>', ('id', 'x'), ('id', 'y')), ('if', ('>', ('id', 'z'), ('id', 'x')), ('id', 'z'), ('id', 'x')), ('if', ('>', ('id', 'z'), ('id', 'y')), ('id', 'z'), ('id', 'y'))), []))

max = (λ(x,y,z). (if (x > y) then (if (z > x) then z else x) else (if (z > y) then z else y)) @ [])

=
	(id,max)
	lambda
		params
			(id,x)
			(id,y)
			(id,z)
		if
			>
				(id,x)
				(id,y)
			if
				>
					(id,z)
					(id,x)
				(id,z)
				(id,x)
			if
				>
					(id,z)
					(id,y)
				(id,z)
				(id,y)
		[]


## Problem Two: AST Evaluator for MyPython 1.2 (20 pts)

Corresponding to the three parts of Problem One, we will develop the eval function in three stages. In addition, we will add some simple error checking as Part D. It is actually a little easier if you do closures first. 

The code below has "Your code here" where you need to develop new code. The entire section for applications has been left out; if you wish to see my solution from HW 04, it is posted online. 

### Part A: Closures
I essentially gave you the code for this in lecture on Tuesday. There are two things to do:

 - When you evaluate a closure <code> ('lambda',&lt;params&gt;, &lt;body&gt;, e)</code>
 you must return a new closure where the current env is added to the rear of the closure environment, i.e., env + e. 
 - When you evaluate an application of a lambda expression to its arguments, you must return the eval of the body of the lambda expression in the new environment consisting of the recursive environment env, the closure environment e, and the new bindings, i.e., env + e + bindings. (The ordering here is important!)

### Part B: Variable parameter and argument lists
Now we need to add code to eval to account for this new kind of application. What you need to 
do is to match up the parameters in the lambda with the arguments in the third component
of the apply tuple. Each of the arguments must be evaluated separately, and
these bindings must be added simultaneously to the environment. 
So

<pre>
('apply', 
         ('lambda', ('params', ('id', 'x'), ('id', 'y') ),                 ('&#42;', ('int', 4),('int', 7)) ),
         ('args',              ('int', 2),  ('&#42;', ('int', 4),('int', 4)) )                                )
</pre>
would result in the bindings <code>[('x', 2),('y',16)]</code> being added to the environment in which
<code> ('*', ('int', 4),('int', 7))</code> is evaluated. 
<p> We will add some error checking to this code in Part D below. 



### Part C: List Data Type
This part will actually take the most thinking and the most coding. There are two things to think about:

 - Cons and nil tuples are not evaluated, since they form "containers" for expressions and values; you will, however, need to evaluate the ASTs stored inside these list ASTs. Therefore, when you evaluate a cons tuple, 
<code>('cons', &lt;exprA&gt;, &lt;exprB&gt;)</code>
you must recursively evaluate the two expressions and return a new cons tuple
<code>('cons', &lt;valA&gt;, &lt;valB&gt;).</code> The evaluation of <code>('nil,)</code> just returns <code>('nil',)</code>, which stops the recursive evaluation of the elements of the list. The result of such an evaluation is to leave the structure of the (possibly nested lists) the same, but to evaluate the expressions contained in the list(s). 

 - There are three built-in functions, presented in the rules 51 - 53, and represented by the tuples <code>('cons',), ('first',), and ('rest',)</code>. <b>Be sure you understand that ('cons',) is NOT a list, as in the previous paragraph, but a function available in MyPython to construct lists.</b> As with <code>('nil,)</code>, integers, floats, etc., when the parameter t in eval is one of these tuples, they evaluate to themselves, you just return t.  HOWEVER, when you encounter them as the function in an apply, you must implement them by manipulating the argument, which must be a list. In this sense, they are "built-in," since they manipulate ASTs.  For example, if the user types in
 <pre>
             cons(1+2,[5])
 </pre>
 then the lexer will return the following list of tuples:
 <pre>
 [('cons',), ('(',), ('int', 1), ('+',), ('int', 2), (',',), ('[',), ('int', 5), (']',), (')',)]
 </pre> and the parser will create the following AST:
 <pre>
      ('apply', ('cons',), ('args', ('+', ('int', 1), ('int', 2)), ('cons', ('int', 5), ('nil',))))
 </pre>
 The evaluation of this AST will result in a new list
  <pre>
      ('cons', ('int',3) ('cons', ('int',5), ('nil',))) ) 
 </pre>
 which is then pretty-printed for the user as
 <pre>
      [3,5]
 </pre>
 Always remember that you must evaluate the arguments to any function before figuring out what the function
 does to the arguments. The ordering of the evaluation in the case of apply is now a little tricky, because
 you may have a variable which is bound to one of these build-in functions (see test case 15). 


The test cases should answer any other questions. 

### Part D: Dynamic Error Checking

We have not worried about error checking too much in this project, but we will do something to avoid
the most egregious misuse of function application. You must deal with two different kinds of errors:

 - The application of a non-lambda expression to arguments: after evaluating the function expression in an apply, it is not a lambda, then you should raise a <code>FunctionTypeException</code> and report the error to the user in the console. 
 
 - A mismatch between the number of arguments and the number of parameters, resulting in an <code>ArgumentMismatchException</code>.
 
The solution sheet for the homework shows a typical interaction, and you should simulate that as closely as possible. 

In [16]:
## Evaluator for ASTs:      

# eval(t,env,memory) takes an abstract syntax tree, an environment, and a global memory and evaluates
# the term in the context of the bindings in env, which is a stack, so
# the first occurrence of the binding is the one that is used; if a variable
# is not found in the env, it is looked for in the global memory; if not there
# an unbound variable exception is raised

# global memory is a dictionary of bindings, e.g., { 'x': 4.5, 'y':-1 }

# bindings are in the form of a tuple (str,value), e.g., ('x',4.5) so
# environment is a list of bindings, e.g., [ ('x',4.5), ('y',-1), ('x',4) ]

# for efficiency, env stack grows from left to right, so push binding on env for
# the recursive call by using env + [binding]

# Use the following for errors with unbound variables

class UnboundVariableException(Exception):
    def __init__(self, name):
        self.name = name

# Your code here for the two additional exceptions
class FunctionTypeException(Exception):
    def __init__(self, name):
        self.name = name
class ArgumentMismatchException(Exception):
    def __init__(self, name):
        self.name = name


# look up a variable id (the string representation, not the whole token) in the
# environment from end of list to beginning; if not there, look in memory, if
# not there, raise an exception:   raise UnboundVariableException(name) 

def lookup(name,env,memory):
    for i in range(len(env)-1,-1,-1):
        if(name == env[i][0]):
            return env[i][1]
    if name in memory:
        return memory[name]
    else:
        raise UnboundVariableException(name)       


def eval(t,env,memory):
    #print(AST_to_string(t) + '\t' + str(env))     # to debug
    
    if t[0] == 'nil':                  # nil is nil!
        return  t
    elif t[0] == 'True':
        return  1
    elif t[0] == 'False':
        return  0
    elif t[0] in ['int','float']:
        return t[1]
    
    # Your code here to deal with ('cons',), ('first',), and ('rest',) as explained above
   
    elif t[0] == 'first':
        return t
    elif t[0] == 'rest':
        return t
    elif t[0] == 'id':                   # identifier, so must look up in env then global memory
        return  lookup(t[1],env,memory)
    elif t[0] == '+':
        return eval(t[1],env,memory) + eval(t[2],env,memory)
    elif t[0] == '*':
        return eval(t[1],env,memory) * eval(t[2],env,memory)
    elif t[0] == '-' and len(t) == 3:
        return eval(t[1],env,memory) - eval(t[2],env,memory)
    elif t[0] == '-' and len(t) == 2:
        return -eval(t[1],env,memory)
    elif t[0] == '/':
        return eval(t[1],env,memory) / eval(t[2],env,memory)
    elif t[0] == '%':
        return eval(t[1],env,memory) % eval(t[2],env,memory)
    elif t[0] == '**':
        return eval(t[1],env,memory) ** eval(t[2],env,memory)
    elif t[0] == 'and':
        res = eval(t[1],env,memory) and eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == 'or':
        res = eval(t[1],env,memory) or eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == 'not':
        res = not eval(t[1],env,memory)
        return 1 if res else 0
    elif t[0] == '<':
        res = eval(t[1],env,memory) < eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == '>':
        res = eval(t[1],env,memory) > eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == '<=':
        res = eval(t[1],env,memory) <= eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == '>=':
        res = eval(t[1],env,memory) >= eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == '==':
        res = eval(t[1],env,memory) == eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == '!=':
        return eval(t[1],env,memory) != eval(t[2],env,memory)
        return 1 if res else 0
    elif t[0] == '=':                    # assignment expression
        val = eval(t[2],env,memory)
        memory[t[1][1]] = val
        return  val
    elif t[0] == 'if':                   # if-then-else
        (_,test,then_exp,else_exp) = t
        return  eval(then_exp,env,memory) if eval(test,env,memory) else eval(else_exp,env,memory) 
    elif t[0] == 'let':                  # let expression, so must extend env with new binding
        (_,(_,(_,var),exp1),exp2) = t
        binding = (var,eval(exp1,env,memory))
        return  eval(exp2,env + [binding],memory)
    elif t[0] == 'lambda':
        (_,var,body,e) = t
        return ('lambda',var,body,env+e)
    elif t[0] == 'apply':
        s = eval(t[1],env,memory)
        s1 = eval(t[2],env,memory)
        if (type(s) != tuple):
            raise FunctionTypeException(t)
        if s == ('cons',):
            return ('cons',s1[1],s1[2])
        elif s == ('first',):
            return (s1[1][1])
        elif s == ('rest',):
            return (s1[1][2])
        elif s[0] == 'lambda':
            args = len(t[2])
            (_,exp1,exp2) = t
            f=eval(exp1,env,memory)
            
#             if len(s) == args:
            
            if (len(f[1]) != args):
                raise ArgumentMismatchException(t)
            elif(len(exp2) ==1):
                (_,name,body,e) = f
                return eval(body,env+e, memory)
            elif len(exp2) == 2:
                (_,(_,name),body,e) = f
                return eval(body,env+e+[(name[1],eval(exp2[1],env,memory))], memory)
            elif len(exp2) == 3:
                (_,(_,name,name2),body,e) = f
                return eval(body,env+e+[(name[1],eval(exp2[1],env,memory)),(name2[1],eval(exp2[2],env,memory))], memory)
            elif len(exp2) == 4:
                (_,(_,name1,name2,name3),body,e) = f
                return eval(body,env+e+[(name1[1],eval(exp2[1],env,memory)), (name2[1],eval(exp2[2],env,memory)),(name3[1],eval(exp2[3],env,memory))], memory)
        else:
            raise FunctionTypeException(t)
#             (_,(_,name),body,e) = f
#             return eval(body,env+e+[(name,eval(exp2,env,memory))], memory)    

    elif t == ('cons',):
        return t
    elif t[0] == 'cons':
        t1 = eval(t[1],env,memory)
        t2 = eval(t[2],env,memory)
        return ('cons',t1,t2)
    elif t[0] == 'args':
        if len(t) ==1:
            return ('args',)
        elif len(t) == 2:
            t1 = eval(t[1],env,memory)
            return ('args', t1)
        elif len(t) == 3:
            t1 = eval(t[1],env,memory)
            t2 = eval(t[2],env,memory)
            return ('args', t1,t2)
        elif len(t) == 4:
            t1 = eval(t[1],env,memory)
            t2 = eval(t[2],env,memory)
            t3 = eval(t[3],env,memory)
            return ('args',t1,t2,t3)
    
    # Your code here to deal with t[0] being 'cons', args', or 'apply'
    # You probably want to deal with 'params' inside the apply, but you could do them
    #     separately in the main list of cases

    else:
        return t


In [17]:
# test 1   (from last time -- should still work!!) 
s = 'let x = 3 in (lambda y : lambda z : x * y + z)(4)'
t = parse(lexer(s),Grammar1_2,table)
print("Should be \n(λ(z). ((x * y) + z) @ [('x', 3), ('x', 3), ('y', 4)])")
print(AST_to_string(eval(t,[],{}))) 

Should be 
(λ(z). ((x * y) + z) @ [('x', 3), ('x', 3), ('y', 4)])
(λ(z). ((x * y) + z) @ [('x', 3), ('x', 3), ('y', 4)])


In [18]:
# test 2  (from last time -- should still work!!!)
s = '(let x = 3 in (lambda y : lambda z : x * y + z)(4))  \
     ((let f = let x = 4 in (lambda y : lambda z : if x < 3 then y * z else y / z)(4) in f(5)) \
   * (lambda x : x * 2) ((lambda x : let y = 1 in x + y) (3)) \
   / (lambda x : x ** 2)((lambda y : y * 2)((lambda x : x + 1)(3))))'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 12.1:")
print(eval(t,[],{}))

Should be 12.1:
12.1


In [19]:
# test 3 
s = '(lambda : 4 + x*y)()'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 22:")
print(eval(t,[('y',3)],{'x' : 6})) 

Should be 22:
22


In [20]:
# test 4 
s = '(lambda x,y : x + y * z)(2,5)'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 12:")
print(eval(t,[('z',2)],{})) 

Should be 12:
12


In [21]:
# test 5 
s = '(lambda x,y,z : a + x + y * z)(2,5,8)'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 43:")
print(eval(t,[('a',1)],{'a' : 6})) 

Should be 43:
43


In [22]:
# test 6 
s = '(lambda a,b,c : lambda d,e : lambda f : lambda : a+b+c+d+e+f)(1,2,3)(4,5)(6)()'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 21:")
print(eval(t,[],{})) 

Should be 21:
21


In [23]:
# test 7 
s = 'let x = 3 in (if x <= 2 then (lambda x : x * 2) else (lambda x : x * 3))(5)'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 15:")
print(eval(t,[],{})) 

Should be 15:
15


In [24]:
# test 8 
s = '(lambda x : (if x > 0 then (lambda x : x * 2) else (lambda x : x * 3)))(4)(5)'
t = parse(lexer(s),Grammar1_2,table)
print("Should be 10:")
print(eval(t,[],{})) 

Should be 10:
10


In [25]:
# test 9 
s = 'cons(1, cons(2, nil))'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('apply', ('cons',), ('args', ('int', 1), ('apply', ('cons',), ('args', ('int', 2), ('nil',)))))")
print(t)
print()
print("Should be: \n[1,2]")
print(AST_to_string(eval(t,[],{}))) 

Should be: 
('apply', ('cons',), ('args', ('int', 1), ('apply', ('cons',), ('args', ('int', 2), ('nil',)))))
('apply', ('cons',), ('args', ('int', 1), ('apply', ('cons',), ('args', ('int', 2), ('nil',)))))

Should be: 
[1,2]
[1,2]


In [26]:
# test 10 
s = '[3,[6,7]]'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('cons', ('int', 3), ('cons', ('cons', ('int', 6), ('cons', ('int', 7), ('nil',))), ('nil',)))")
print(t)
print("\nShould be: \n[3,[6,7]]")
print(AST_to_string(eval(t,[],{}))) 

Should be: 
('cons', ('int', 3), ('cons', ('cons', ('int', 6), ('cons', ('int', 7), ('nil',))), ('nil',)))
('cons', ('int', 3), ('cons', ('cons', ('int', 6), ('cons', ('int', 7), ('nil',))), ('nil',)))

Should be: 
[3,[6,7]]
[3,[6,7]]


In [27]:
# test 9 
s = '[3,4+3,8/2]'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('cons', ('int', 3), ('cons', ('+', ('int', 4), ('int', 3)), ('cons', ('/', ('int', 8), ('int', 2)), ('nil',))))")
print(t)
print("\nShould be: \n[3,7,4.0]")
print(AST_to_string(eval(t,[],{}))) 

Should be: 
('cons', ('int', 3), ('cons', ('+', ('int', 4), ('int', 3)), ('cons', ('/', ('int', 8), ('int', 2)), ('nil',))))
('cons', ('int', 3), ('cons', ('+', ('int', 4), ('int', 3)), ('cons', ('/', ('int', 8), ('int', 2)), ('nil',))))

Should be: 
[3,7,4.0]
[3,7,4.0]


In [28]:
# test 10 
s = 'let x = 1 in cons(1, [x+1,3])'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('let', ('=', ('id', 'x'), ('int', 1)), ('apply', ('cons',), ('args', ('int', 1), ('cons', ('+', ('id', 'x'), ('int', 1)), ('cons', ('int', 3), ('nil',))))))")
print(t)
print("\nShould be: \n[1,2,3]")
print(AST_to_string(eval(t,[],{})))

Should be: 
('let', ('=', ('id', 'x'), ('int', 1)), ('apply', ('cons',), ('args', ('int', 1), ('cons', ('+', ('id', 'x'), ('int', 1)), ('cons', ('int', 3), ('nil',))))))
('let', ('=', ('id', 'x'), ('int', 1)), ('apply', ('cons',), ('args', ('int', 1), ('cons', ('+', ('id', 'x'), ('int', 1)), ('cons', ('int', 3), ('nil',))))))

Should be: 
[1,2,3]
[1,2,3]


In [29]:
# test 11 
s = 'cons(1, [])'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('apply', ('cons',), ('args', ('int', 1), ('nil',)))")
print(t)
print("\nShould be: \n[1]")
print(AST_to_string(eval(t,[],{})))

Should be: 
('apply', ('cons',), ('args', ('int', 1), ('nil',)))
('apply', ('cons',), ('args', ('int', 1), ('nil',)))

Should be: 
[1]
[1]


In [30]:
# test 12 
s = 'first( [1,2] )'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('apply', ('first',), ('args', ('cons', ('int', 1), ('cons', ('int', 2), ('nil',)))))")
print(t)
print("\nShould be: \n1")
print(AST_to_string(eval(t,[],{})))

Should be: 
('apply', ('first',), ('args', ('cons', ('int', 1), ('cons', ('int', 2), ('nil',)))))
('apply', ('first',), ('args', ('cons', ('int', 1), ('cons', ('int', 2), ('nil',)))))

Should be: 
1
1


In [31]:
# test 13
s = 'rest( [1,2,3] )'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n('apply', ('rest',), ('args', ('cons', ('int', 1), ('cons', ('int', 2), ('cons', ('int', 3), ('nil',))))))")
print(t)
print("\nShould be: \n[2,3]")
print(AST_to_string(eval(t,[],{})))

Should be: 
('apply', ('rest',), ('args', ('cons', ('int', 1), ('cons', ('int', 2), ('cons', ('int', 3), ('nil',))))))
('apply', ('rest',), ('args', ('cons', ('int', 1), ('cons', ('int', 2), ('cons', ('int', 3), ('nil',))))))

Should be: 
[2,3]
[2,3]


In [32]:
# test 14 
s = 'rest(cons(1, cons(2, first( [[3,4],5] ))))'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n[2,3,4]")
print(AST_to_string(eval(t,[],{})))

Should be: 
[2,3,4]
[2,3,4]


In [33]:
# test 15 
s = '(if 13 < 8 then first else rest)([1,2,3])'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n[2,3]")
print(AST_to_string(eval(t,[],{})))

Should be: 
[2,3]
[2,3]


In [34]:
# test 16 
s = '(lambda x,y : cons(x,rest(y)))([1,2],[3,4])'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n[[1,2],4]")
print(AST_to_string(eval(t,[],{})))

Should be: 
[[1,2],4]
[[1,2],4]


In [35]:
# test 17 
s = 'rest(rest( [ 1, 2 ]))'
t = parse(lexer(s),Grammar1_2,table)
print("Should be [] or: \nnil")
print(AST_to_string(eval(t,[],{})))

Should be [] or: 
nil
nil


In [36]:
# test 18
s = 'rest(rest( [ 1, 2 ])) == []'
t = parse(lexer(s),Grammar1_2,table)
print("Should be: \n1")
print(AST_to_string(eval(t,[],{})))

Should be: 
1
1


## Problem 3 (5 pts)

Now we will do some programming! You must modify the console code below to do three things:

 - Catch the FunctionTypeException and print out the offending function or the expression
 - Catch the ArgumentMismatchException print out the offending expression
 - Show that you can reproduce the Python code in the next cell in MyPython syntax (e.g., L[0] must be
   replaced by first(L), etc.)
   
I strongly suggest that you write your functions out and cut and paste them into the input buffer; an example is given at the end of the notebook. 
   
A print out of the solution has been given in the PDF solution sheet online. You must reproduce this as well as possible and submit your notebook with this output for this problem.



In [ ]:
# This Python code shows how map and reduce -- two famous examples of the functional programming paradigm -- work. 

def times2(x):
    return x * 2

def even(x) :
    return x % 2 == 0

def max(x,y):
    if x > y:
        return x
    else:
        return y
    
# Map takes a function and a list and returns a new list where function f has been applied to
# every member of the list
    
def map(f,L):
    if L == []:
        return []
    else:
        return [f(L[0])] + map(f,L[1:])

# Filter takes a function from ints to booleans and a list of ints and reduces the sublist
# of all those elements for which the function returns True

def filter(f,L):
    if L == []:
        return []
    elif f(L[0]):
        return [L[0]] + filter(f,L[1:])
    else:
        return filter(f,L[1:])

# reduce takes a two-argument function f and a list L [ x1, x2, x3, ..., xn ] and
# returns  f(x1,f(x2,f(x3, ..  f(x(n-1),xn) ... ))) ;  if L has one element, it is returned. 

def reduce(f,L):
    if len(L) == 1:
        return L[0]
    else:
        return f(L[0],reduce(f,L[1:]))
    
lst = [2,3,6,4]
    
print(times2(4))
print(str(even(4)) + "\t Note: your MyPython function will return 1")
print(max(2,3))

print(map(times2, lst))  
print(filter(even,lst))  
print(reduce(max, lst))  
print(reduce(max, map(times2,filter(even,lst))))    

8
True	 Note: your MyPython function will return 1
3
[4, 6, 12, 8]
[2, 6, 4]
6
12


In [ ]:
# Basic Interpreter

# use the following as a framework for imitating what I provide in the solution pdf
# to catch an exception, use something like this:
#            try:
#                print("Out["+str(lineNum)+"]: " + str(eval(t,[])))
#            except UnboundVariableException as n:
#                print("Unbound Variable: " + str(n))

lineNum = 1

global_memory = { }

s = input("\nMyPython 1.2\nInput equation or type \"quit\" to quit:\n\nIn ["+str(lineNum)+"]: ")

while True:
    if s == "":
        pass
    elif s == 'quit':      
        print("\nBye!")
        break
    elif s == "global":
        print(global_memory)
    else:
        t = parse(lexer(s), Grammar1_2, table, False)
        
        if t == None:
            print("\nError, try again!")
        else:
            try:
                print("Out["+str(lineNum)+"]: " + AST_to_string(eval(t,[],global_memory)))
            except UnboundVariableException as n:
                print("Unbound Variable: " + str(s))
                
            except FunctionTypeException as n:
                print("Type Error: " + str(s))
                
            except ArgumentMismatchException as n:
                print("Argument Mismatch Error: " + str(s))
            # Your code here to deal with the other two exceptions
                      

    lineNum += 1
    s = input("\nIn ["+str(lineNum)+"]: ")

 

In [ ]:
# cut and paste this into the input box in the console above:  
# def times2(x) : return x * 2